# Export project data from Prod to QA for pipeline testing

Production PID's: 100, 278

QA PID's: Copies of above PID's: 75, 74

In [177]:
import redcap

In [179]:
#API URL and API keys import
%run -i -n "credentials.py"

print(dev_url)
print(prod_url)
print(["dev_pid100", "dev_pid278", "prod_pid100", "prod_pid278"])

https://qa-redcap.doh.wa.gov/api/
https://redcap.doh.wa.gov/api/
['dev_pid100', 'dev_pid278', 'prod_pid100', 'prod_pid278']


In [180]:
#Create Project objects
prod100 = redcap.Project(prod_url, prod_pid100)
prod278 = redcap.Project(prod_url, prod_pid278)

dev100 = redcap.Project(dev_url, dev_pid100)
dev278 = redcap.Project(dev_url, dev_pid278)

## Explore PID100 Dataset, Export Text Fields to QA

In [4]:
prod100_df = prod100.export_records(format_type = "df", raw_or_label = "raw")
p100meta = prod100.export_metadata(format_type = "df")
p100meta.drop("record_id", inplace = True) #remove "record_id" from metadata (this is REDCap index)

C:\Users\AXG5303\Miniconda3\envs\transform22\lib\site-packages\redcap\methods\base.py:137: DtypeWarning: Columns (397) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(buf, **df_kwargs)


In [5]:
#import 500 rows of all fields that are "text" type
filt_txt = p100meta["field_type"] == "text"
fields_txt = set(p100meta[filt_txt].index)

In [6]:
#verify txt fields are the same length with the intersection of txt fields
print(len(fields_txt))
print(len(fields_txt & set(prod100_df.columns)))

#verify that all txt fields exist in dataset 
common = fields_txt & set(prod100_df.columns)
common == fields_txt

257
257


True

In [7]:
#first 500 rows, and all text fields
df_import_txt = prod100_df.iloc[0:500,:].loc[:, list(fields_txt)].copy()

In [8]:
#Convert all float type columns that are defined as text fields in redcap, into Int64 type
floating = df_import_txt.dtypes == np.floating
df_import_txt.loc[:, floating] = df_import_txt.loc[:, floating].astype("Int64")

In [9]:
dev100.import_records(df_import_txt, import_format = "df")

{'count': 500}

## Explore PID100 Dataset, Export radio fields to QA

In [10]:
p100meta["field_type"].value_counts()

text           257
descriptive     83
radio           57
file            30
dropdown        27
yesno           24
notes            8
checkbox         5
Name: field_type, dtype: int64

In [87]:
meta_groups = p100meta.groupby("field_type")
fields_radio = meta_groups.get_group("radio").index

#verify all columns exist within dataframe
print(len(fields_radio))
print(len(set(fields_radio) & set(prod100_df.columns)))

57
57


In [78]:
df_import_radio = prod100_df.iloc[0:500,:].loc[:, list(fields_radio)].copy()

#changing float dtypes to Int
floating_radio = df_import_radio.dtypes == np.floating
df_import_radio.loc[:, floating_radio] = df_import_radio.loc[:, floating_radio].astype("Int64")

In [82]:
dev100.import_records(df_import_radio, import_format = "df")

{'count': 500}

## Explore PID100 Dataset, Export radio fields to QA

file fields are exported as str names of the uploaded files. Will not export these fields.

In [98]:
fields_file = meta_groups.get_group("file").index

In [104]:
print(len(fields_file))
print(len(set(prod100_df.columns) & set(fields_file)))

30
30


In [ ]:
prod100_df.loc[:, list(fields_file)]

## Explore PID100 Dataset, Export dropdown fields to QA



In [106]:
fields_dropdown = meta_groups.get_group("dropdown").index

In [108]:
print(len(fields_dropdown))
print(len(set(prod100_df.columns) & set(fields_dropdown)))

27
27


In [154]:
df_import_dropdown = prod100_df.iloc[0:500,:].loc[:, list(fields_dropdown)].copy()

#changing float dtypes to Int
floating_dropdown = df_import_dropdown.dtypes == np.floating
df_import_dropdown.loc[:, floating_dropdown] = df_import_dropdown.loc[:, floating_dropdown].astype("Int64")

In [155]:
#renaming values from raw to coded, why do some fields import raw values and some fields import on labels???
org_wa_county = {"Pend_Orielle": "Pend_Oreille"}
df_import_dropdown["org_wa_county"] = df_import_dropdown["org_wa_county"].replace(org_wa_county)

df_import_dropdown["org_wa_county"].value_counts()


King            131
Pierce           83
Spokane          41
Snohomish        39
Thurston         23
Kitsap           23
Whatcom          16
Yakima           14
Lewis            12
Skagit           10
Benton            8
Okanogan          8
Grant             8
Clark             8
Clallam           7
Pacific           6
Chelan            5
Island            5
Grays_Harbor      5
Mason             5
Klickitat         4
Lincoln           4
Whitman           3
Kittitas          3
Stevens           2
Franklin          2
Walla_Walla       2
Jefferson         2
San_Juan          2
Cowlitz           1
Wahkiakum         1
Ferry             1
Skamania          1
Pend_Oreille      1
Asotin            1
Adams             1
Columbia          1
Douglas           1
Name: org_wa_county, dtype: int64

In [159]:
df_import_dropdown[["org_wa_county","org_cmo_county", "org_ceo_county"]] = df_import_dropdown[["org_wa_county","org_cmo_county", "org_ceo_county"]].replace(org_wa_county)

In [163]:
df_import_dropdown[["org_wa_county","org_cmo_county", "org_ceo_county"]].value_counts()

org_wa_county  org_cmo_county  org_ceo_county
King           King            King              36
Pierce         Pierce          Pierce            16
Spokane        Spokane         Spokane           13
Snohomish      Snohomish       Snohomish         11
Whatcom        Whatcom         Whatcom            9
Yakima         Yakima          Yakima             8
Thurston       Thurston        Thurston           8
Skagit         Skagit          Skagit             7
Kitsap         Kitsap          Kitsap             6
Clallam        Clallam         Clallam            5
Grant          Grant           Grant              5
Lewis          Lewis           Lewis              5
Chelan         Chelan          Chelan             4
Island         Island          Island             4
Whitman        Whitman         Whitman            3
Lincoln        Lincoln         Lincoln            3
Pacific        Pacific         Pacific            3
King           Snohomish       Snohomish          2
Pierce         Kin

In [162]:
dev100.import_records(df_import_dropdown, import_format = "df")

{'count': 500}

## Explore PID100 Dataset, Export yes/no fields to QA


In [167]:
fields_yesno = meta_groups.get_group("yesno").index

#verify all columns exist within dataframe
print(len(fields_yesno))
print(len(set(fields_yesno) & set(prod100_df.columns)))

24
24


In [171]:
df_import_yesno = prod100_df.iloc[0:500,:].loc[:, list(fields_yesno)].copy()

In [174]:
df_import_yesno = df_import_yesno.astype("Int64")

In [186]:
dev100.import_records(df_import_yesno, import_format = "df")

{'count': 500}

## Explore PID100 Dataset, Export Notes fields to QA

In [187]:
fields_notes = meta_groups.get_group("notes").index

#verify all columns exist within dataframe
print(len(fields_notes))
print(len(set(fields_notes) & set(prod100_df.columns)))

8
8


In [261]:
df_import_notes = prod100_df.iloc[0:500,:].loc[:, list(fields_notes)].copy()

In [265]:
dev100.import_records(df_import_notes, import_format = "df")

{'count': 500}

## Explore PID100 Dataset, Export Notes and Checkboxes fields to QA

In [250]:
fields_checkbox_orig = meta_groups.get_group("checkbox").index

#verify all columns exist within dataframe
print(len(fields_checkbox_orig))
print(len(set(fields_checkbox_orig) & set(prod100_df.columns)))

5
0


In [257]:
fields_checkbox = list(fields_checkbox_orig + "__")
a = "|".join(fields_checkbox)
b = prod100_df.columns.str.contains(a)

df_import_checkbox = prod100_df.iloc[0:500,:].loc[:, list(prod100_df.columns[b])].copy()

In [266]:
len(df_import_checkbox.columns)

56

In [259]:
dev100.import_records(df_import_checkbox, import_format = "df")

{'count': 500}

## Evaluate All Imported Fields

All remaining after importing by category type (metadata)

In [269]:
imported_dfs = [df_import_checkbox, df_import_dropdown, df_import_notes, df_import_radio, df_import_txt, df_import_yesno]

#list of lists for all imported fields
imported_fields = []
for i in imported_dfs:
    clms_list = list(i.columns)
    imported_fields.append(clms_list)
    


In [274]:

all_imported_fields = [item for sublist in imported_fields for item in sublist]

In [276]:
len(all_imported_fields)

429

In [292]:
#All standard columns except for fields that contain files
prod100_clms = set(prod100_df.columns) - set(fields_file)
len(prod100_clms)

441

In [295]:
#are all imported fields contained in the standard columns?
common_0 = set(prod100_clms) & set(all_imported_fields)
common_0 == set(all_imported_fields)

True

In [294]:
len(set(prod100_clms) & set(all_imported_fields))

429

In [299]:
#what columns make up the difference? 
diff = prod100_clms - set(all_imported_fields)

In [300]:
diff

{'agreement_and_signatures_complete',
 'agreement_prescreen_complete',
 'facility_information_complete',
 'facility_storage_complete',
 'inquiry_enrollment_survey_complete',
 'inquiry_review_form_complete',
 'isa_approval_complete',
 'list_of_providers_complete',
 'organization_information_complete',
 'prescreening_survey_complete',
 'vaccine_management_team_review_complete',
 'waiis_team_review_complete'}

In [304]:
prod100_df.loc[:,list(diff)].dtypes

vaccine_management_team_review_complete    int64
agreement_prescreen_complete               int64
waiis_team_review_complete                 int64
organization_information_complete          int64
list_of_providers_complete                 int64
inquiry_enrollment_survey_complete         int64
facility_information_complete              int64
isa_approval_complete                      int64
prescreening_survey_complete               int64
agreement_and_signatures_complete          int64
inquiry_review_form_complete               int64
facility_storage_complete                  int64
dtype: object

In [305]:
df_import_complete = prod100_df.iloc[0:500].loc[:,list(diff)]

In [306]:
df_import_complete

,vaccine_management_team_review_complete,agreement_prescreen_complete,waiis_team_review_complete,organization_information_complete,list_of_providers_complete,inquiry_enrollment_survey_complete,facility_information_complete,isa_approval_complete,prescreening_survey_complete,agreement_and_signatures_complete,inquiry_review_form_complete,facility_storage_complete
record_id,,,,,,,,,,,,
1,2,2,2,2,2,0,2,2,2,2,0,2
2,2,2,2,2,2,0,2,2,2,2,0,2
3,2,2,2,2,2,0,2,2,2,2,0,2
4,2,2,2,2,2,0,2,2,2,2,0,2
5,2,2,2,2,2,0,2,0,2,2,0,2
6,2,2,2,2,2,0,2,2,2,2,0,2
7,2,2,2,2,2,0,2,2,2,2,0,2
8,2,2,2,2,2,0,2,0,2,2,0,2
9,2,2,2,2,2,0,2,0,2,2,0,2
